In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import cv2  
from tqdm import tqdm

from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76

from collections import defaultdict

In [2]:
train_path = "data/train"
test_path = "data/test"

In [3]:
train = os.listdir(train_path)
test = os.listdir(test_path)

In [4]:
train_image = [cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB)  for image in train ]

In [5]:
test_image = [cv2.cvtColor(cv2.imread(os.path.join(test_path, image)), cv2.COLOR_BGR2RGB)  for image in test ]

In [6]:
sift = cv2.SIFT_create() 

In [7]:
kp_train = []
des_trian = []
for image in tqdm(train_image):
    kp, des = sift.detectAndCompute(image,None)
    kp_train.append(kp)
    des_trian.append(des)

100%|██████████| 7500/7500 [07:02<00:00, 17.77it/s]


In [8]:
kp_test = []
des_test = []
for image in tqdm(test_image):
    kp, des = sift.detectAndCompute(image,None)
    kp_test.append(kp)
    des_test.append(des)

100%|██████████| 1200/1200 [01:04<00:00, 18.68it/s]


In [9]:
index_params = dict(algorithm = 1, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)

In [10]:
for i in range(len(des_trian)):
    flann.add([des_trian[i]])

In [11]:
# Train tree structure FLANN 
flann.train()

In [12]:
print(len(flann.getTrainDescriptors()))

7500


# FLANN Match

In [ ]:
result = {}
for i in tqdm(range(len(test))):
    matches = flann.match(des_test[i])
    matches_dict = defaultdict(lambda : 0)
    for f in matches:
        matches_dict[f.imgIdx]+=1
    temp = sorted(matches_dict.items(),key=lambda x:x[1],reverse=True)
    result[test[i]] = temp

In [ ]:
result_index = []
for key in result.keys():
    try:
        result_index.append(result[key][0][0])
    # if the detector is None, the FLANN can not find any match item, return None
    except:
        result_index.append(None)

In [ ]:
train_data = pd.read_csv("data/train.csv")

In [ ]:
cnn = pd.read_csv("result/CNN_baseline.csv")

In [ ]:
# Using the baseline result to replace the None
x_coor = []
y_coor = []
for i in range(len(result_index)):
    index = result_index[i]
    if index:
        x_coor.append(train_data.iloc[index][["x"]][0])
        y_coor.append(train_data.iloc[index][["y"]][0])
    else:
        x_coor.append(cnn.iloc[i][["x"]][0])
        y_coor.append(cnn.iloc[i][["y"]][0])


In [ ]:
submit = pd.read_csv("data/imagenames.csv")
submit["x"] = x_coor
submit["y"] = y_coor

In [ ]:
submit.to_csv("result/sift_match.csv",index = False)

# FLANN KNN_match

In [41]:
result_knn = {}
for i in tqdm(range(len(test))):
    knn_matches = flann.knnMatch(des_test[i],k=2)
    matches_dict = defaultdict(lambda : 0)
    for m,n in knn_matches:
        if m.distance < 0.6 *n.distance: 
            matches_dict[m.imgIdx]+=1
    temp = sorted(matches_dict.items(),key=lambda x:x[1],reverse=True)
    result_knn[test[i]] = temp

100%|██████████| 1200/1200 [08:46<00:00,  2.28it/s]


In [42]:
result_index = []
for key in result_knn.keys():
    try:
        result_index.append(result_knn[key][0][0])
    except:
        result_index.append(None)

In [43]:
train_data = pd.read_csv("data/train.csv")

In [44]:
cnn = pd.read_csv("result/CNN_baseline.csv")

In [45]:
x_coor = []
y_coor = []
for i in range(len(result_index)):
    index = result_index[i]
    if index:
        x_coor.append(train_data.iloc[index][["x"]][0])
        y_coor.append(train_data.iloc[index][["y"]][0])
    else:
        x_coor.append(cnn.iloc[i][["x"]][0])
        y_coor.append(cnn.iloc[i][["y"]][0])

In [47]:
submit = pd.read_csv("data/imagenames.csv")
submit["x"] = x_coor
submit["y"] = y_coor

In [40]:
submit.to_csv("result/sift_KNNmatch0.6.csv",index = False)